### Instalar dependecias

In [ ]:
%matplotlib inline

In [ ]:
%pip install numpy pandas matplotlib scikit-learn notebook ipython

---
### Aquisição e Limpeza dos Dados

Nesta etapa, carregamos o dataset weatherHistory.csv (dados meteorológicos históricos). Este conjunto de dados contém registros horários de condições meteorológicas na região de Szeged, Hungria, entre 2006 e 2016 (disponível no Kaggle). As colunas incluem data/hora, resumo do tempo, tipo de precipitação, temperatura (°C), temperatura aparente (°C), umidade, velocidade e direção do vento, visibilidade, cobertura de nuvens e pressão atmosférica

In [ ]:
import pandas as pd

# Leitura dos dados
df = pd.read_csv('weatherHistory.csv')

# Visão geral das primeiras linhas (amostra dos dados)
print(df.head(5))  # Exibe as primeiras 5 linhas do dataset

# Remover coluna irrelevante (Loud Cover) e Daily Summary que não será usada
df.drop(['Loud Cover', 'Daily Summary'], axis=1, inplace=True)

# Tratar valores ausentes em 'Precip Type': substituir NaN por 'none'
df['Precip Type'].fillna('none', inplace=True)

# Remover linhas duplicadas completas, se houver
df.drop_duplicates(inplace=True)

# Conversão da coluna de data para datetime
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])

# Conferir resultado da limpeza
print("Dados após limpeza:")
print(f"Total de registros: {len(df)}")
print("Valores nulos por coluna:")
print(df.isnull().sum())


---
### Análise Exploratória e Visualização de Dados

Com os dados limpos, prosseguimos para a análise exploratória (Exploratory Data Analysis - EDA) para entender melhor os dados e verificar nossas hipóteses iniciais.



In [ ]:


# Estatísticas descritivas das principais variáveis numéricas
print(df[['Temperature (C)', 'Apparent Temperature (C)', 'Humidity']].describe())

# Calcular correlação entre temperatura real, aparente, umidade e vento
corr_matrix = df[['Temperature (C)', 'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)']].corr()
print("Correlação entre variáveis selecionadas:")
print(corr_matrix)


In [ ]:
# Bloco: Histogramas de Temperature e Apparent Temperature
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

plt.subplot(1, 2, 1)
plt.hist(df['Temperature (C)'], bins=30)
plt.title('Distribuição da Temperatura (°C)')
plt.xlabel('Temperature (C)')
plt.ylabel('Frequência')

plt.subplot(1, 2, 2)
plt.hist(df['Apparent Temperature (C)'], bins=30)
plt.title('Distribuição da Temperatura Aparente (°C)')
plt.xlabel('Apparent Temperature (C)')

plt.tight_layout()
plt.show()


In [ ]:
# Bloco: Scatter Temperature x Apparent Temperature, colorido por Humidity
plt.figure(figsize=(7,7))
sc = plt.scatter(
    df['Temperature (C)'],
    df['Apparent Temperature (C)'],
    c=df['Humidity'],
    alpha=0.5
)
plt.plot([-30,45], [-30,45], 'k--', lw=1)  # linha y=x
plt.xlabel('Temperature (C)')
plt.ylabel('Apparent Temperature (C)')
plt.title('Temp Real vs. Temp Aparente (cor = Umidade)')
cbar = plt.colorbar(sc)
cbar.set_label('Humidity')
plt.show()


In [ ]:
# Bloco: Bar chart com contagem de cada categoria de Summary
summary_counts = df['Summary'].value_counts().head(20)  # top 20
plt.figure(figsize=(10,6))
summary_counts.plot(kind='bar')
plt.title('Top 20 Resumos do Tempo Mais Frequentes')
plt.xlabel('Summary')
plt.ylabel('Contagem')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
# Bloco: Boxplot do desvio da temperatura aparente em relação à real, por Precip Type
df['delta_temp'] = df['Apparent Temperature (C)'] - df['Temperature (C)']

plt.figure(figsize=(6,5))
df.boxplot(column='delta_temp', by='Precip Type', grid=False)
plt.title('Delta Temp (Apparente – Real) por Tipo de Precipitação')
plt.suptitle('')
plt.xlabel('Precip Type')
plt.ylabel('Delta Temperature (°C)')
plt.show()


---
### Modelagem Preditiva e Validação

Compreendida a estrutura dos dados, partimos para a modelagem. Nosso alvo (variável dependente) é Apparent Temperature (C) e usaremos como features (variáveis explicativas) as colunas selecionadas: Temperature (C), Humidity, Summary e Precip Type.


In [ ]:
from sklearn.model_selection import train_test_split



# Seleciona colunas de entrada (X) e alvo (y) para o modelo
features = ['Temperature (C)', 'Humidity', 'Summary', 'Precip Type']
target = 'Apparent Temperature (C)'

# Aplicar One-Hot Encoding nas variáveis categóricas
df_model = df[features + [target]].copy()
df_model = df_model.fillna({'Precip Type': 'none'})  # garantir nenhum NaN em Precip Type
# Converte categorias em dummies
df_model = pd.get_dummies(df_model, columns=['Summary', 'Precip Type'], drop_first=True)

X = df_model.drop(target, axis=1)
y = df_model[target]

# Dividir em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Tamanho treino: {X_train.shape[0]} exemplos, teste: {X_test.shape[0]} exemplos")


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Modelo 1: Regressão Linear
linreg = LinearRegression()
linreg.fit(X_train, y_train)  # Treina o modelo nos dados de treino

# Predição no conjunto de teste
y_pred_lin = linreg.predict(X_test)

# Avaliação do modelo linear
mse_lin = mean_squared_error(y_test, y_pred_lin)
rmse_lin = mse_lin ** 0.5
r2_lin = r2_score(y_test, y_pred_lin)
print(f"Desempenho Regressão Linear - RMSE: {rmse_lin:.3f} °C, R²: {r2_lin:.3%}")

# Modelo 2: Random Forest Regressor (exemplo de modelo de árvore de decisão em conjunto)
rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)
rf.fit(X_train, y_train)

# Predição no teste
y_pred_rf = rf.predict(X_test)

# Avaliação do Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = mse_rf ** 0.5
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Desempenho Random Forest - RMSE: {rmse_rf:.3f} °C, R²: {r2_rf:.3%}")




In [ ]:
# Bloco: Scatter Predito vs. Real para o melhor modelo
y_pred = y_pred_rf  # use o modelo que preferir (rf ou linreg)
plt.figure(figsize=(7,7))
plt.scatter(y_test, y_pred, alpha=0.4)
plt.plot([-30,45], [-30,45], 'r--', lw=1)
plt.xlabel('Temperatura Aparente Real (C)')
plt.ylabel('Temperatura Aparente Prevista (C)')
plt.title('Real vs. Previsto (Random Forest)')
plt.show()
